# 회원탈퇴를 예측해보자.

### 이용데이터 수정

In [27]:
import pandas as pd

customer = pd.read_csv("./data_4/customer_join.csv")
customer['end_date'].value_counts()

end_date
2019-02-28    149
2019-01-31    129
2018-05-31    125
2018-04-30    121
2018-08-31    113
2018-12-31    111
2019-03-31    111
2018-06-30    106
2018-09-30    104
2018-07-31    102
2018-10-31     90
2018-11-30     89
Name: count, dtype: int64

In [11]:
uselog_months = pd.read_csv("./data_4/use_log_month.csv").drop(columns = 'Unnamed: 0')
uselog_months

,year_month,customer_id,count
0,201804,AS002855,4
1,201804,AS009013,2
2,201804,AS009373,3
3,201804,AS015315,6
4,201804,AS015739,7
...,...,...,...
36837,201903,TS995853,8
36838,201903,TS998593,8
36839,201903,TS999079,3
36840,201903,TS999231,6


In [12]:
year_months = list(uselog_months['year_month'].unique())
uselog = pd.DataFrame()
for i in range(1, len(year_months)): # 201805부터
    tmp = uselog_months.loc[uselog_months["year_month"] == year_months[i]]
    tmp.rename(columns = {'count':'count_0'},inplace = True)
    tmp_before = uselog_months.loc[uselog_months['year_month'] == year_months[i-1]]
    del tmp_before['year_month'] # 201804부터 (전달부터)
    tmp_before.rename(columns = {'count':'count_1'},inplace = True)
    tmp = pd.merge(tmp, tmp_before, on = 'customer_id', how = 'left')
    uselog = pd.concat([uselog, tmp],ignore_index = True)
uselog

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_9936\465257005.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp.rename(columns = {'count':'count_0'},inplace = True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_9936\465257005.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_before.rename(columns = {'count':'count_1'},inplace = True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_9936\465257005.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

,year_month,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0
...,...,...,...,...
33846,201903,TS995853,8,11.0
33847,201903,TS998593,8,7.0
33848,201903,TS999079,3,2.0
33849,201903,TS999231,6,6.0


### 2. 탈퇴 전월의 탈퇴 고객데이터 작성

In [35]:
from dateutil.relativedelta import relativedelta

exit_customer = customer.loc[customer['is_deleted']==1]
exit_customer['exit_date'] = None
exit_customer['end_date'] = pd.to_datetime(exit_customer['end_date'])

for i in range(len(exit_customer)):
    exit_customer['exit_date'].iloc[i] = exit_customer['end_date'].iloc[i] - relativedelta(months=1)

exit_customer['year_month'] = pd.to_datetime(exit_customer['exit_date']).dt.strftime('%Y%m')
uselog['year_month'] = uselog['year_month'].astype(str)

exit_uselog = pd.merge(uselog, exit_customer, on = ['customer_id','year_month'], how = 'left')
print(len(uselog))
exit_uselog.head()


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_9936\960562442.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exit_customer['exit_date'] = None
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_9936\960562442.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exit_customer['end_date'] = pd.to_datetime(exit_customer['end_date'])
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_9936\960562442.py:8: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are s

33851


,year_month,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
# 결측치가 없는 데이터(탈퇴한 데이터)
exit_uselog = exit_uselog.dropna(subset=['name'])
len(exit_uselog)

1104

### 3. 지속회원의 데이터 작성

In [44]:
conti_customer = customer.loc[customer['is_deleted']==0]
conti_uselog = pd.merge(uselog, conti_customer, on=['customer_id'],how = 'left')
conti_uselog = conti_uselog.dropna(subset=['name'])
len(conti_uselog)

27422

현재 데이터가 탈퇴회원은 1104이고, 지속회원은 27422명이라 데이터 불균형이기 때문에 데이터를 섞고 중복을 제거하는 방법을 사용한다.

In [61]:
conti_uselog = conti_uselog.sample(frac= 1).reset_index(drop=True) # 무작위로 섞기
conti_uselog = conti_uselog.drop_duplicates(subset="customer_id") # 중복행 제거
len(conti_uselog)

2842

In [62]:
# 결합
predict_data = pd.concat([conti_uselog, exit_uselog],ignore_index=True)
predict_data

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_9936\2448481541.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predict_data = pd.concat([conti_uselog, exit_uselog],ignore_index=True)


,year_month,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201808,GD885447,4,6.0,XXXX,C02,F,2017-07-01,NaT,CA2,...,7500.0,0_입회비반액할인,6.083333,6.0,9.0,3.0,1.0,2019-04-30,21.0,NaN
1,201807,HI518408,7,4.0,XXXX,C01,M,2015-12-01,NaT,CA1,...,10500.0,2_일반,5.333333,5.0,8.0,3.0,1.0,2019-04-30,40.0,NaN
2,201806,GD406715,4,5.0,XXX,C02,F,2016-03-01,NaT,CA1,...,7500.0,2_일반,5.416667,5.0,7.0,3.0,1.0,2019-04-30,37.0,NaN
3,201901,GD659982,9,10.0,XXXXX,C02,F,2018-06-10,NaT,CA2,...,7500.0,0_입회비반액할인,7.800000,7.5,10.0,5.0,1.0,2019-04-30,10.0,NaN
4,201811,HD184011,7,6.0,XXXXXX,C02,F,2017-05-01,NaT,CA1,...,7500.0,2_일반,6.166667,6.0,8.0,4.0,1.0,2019-04-30,23.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3941,201902,TS645212,4,2.0,XXXX,C03,F,2018-03-01,2019-03-31,CA1,...,6000.0,2_일반,4.500000,4.5,7.0,1.0,0.0,2019-03-31,12.0,2019-02-28 00:00:00
3942,201902,TS741703,5,6.0,XXXX,C03,M,2018-12-08,2019-03-31,CA3,...,6000.0,1_입회비무료,6.250000,6.0,8.0,5.0,0.0,2019-03-31,3.0,2019-02-28 00:00:00
3943,201902,TS859258,1,3.0,XXXXX,C02,F,2018-12-07,2019-03-31,CA3,...,7500.0,1_입회비무료,2.500000,2.0,5.0,1.0,0.0,2019-03-31,3.0,2019-02-28 00:00:00
3944,201902,TS886985,5,3.0,XXX,C02,F,2018-03-01,2019-03-31,CA1,...,7500.0,2_일반,4.250000,4.0,7.0,2.0,1.0,2019-03-31,12.0,2019-02-28 00:00:00


### 4. 예측할 달의 재적기간을 작성

In [65]:
for i in range(len(predict_data)):
    now = predict_data['now_date'][i]
    start = predict_data['start_date'][i]
    
    if pd.notna(now) and pd.notna(start):
        delta = relativedelta(now, start)
        predict_data.at[i, 'period'] = int(delta.years * 12 + delta.months)
    else:
        predict_data.at[i, 'period'] = None  # 혹은 0 또는 np.nan

In [66]:
predict_data

,year_month,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,period,now_date
0,201808,GD885447,4,6.0,XXXX,C02,F,2017-07-01,NaT,CA2,...,6.083333,6.0,9.0,3.0,1.0,2019-04-30,21.0,NaN,13,2018-08-01
1,201807,HI518408,7,4.0,XXXX,C01,M,2015-12-01,NaT,CA1,...,5.333333,5.0,8.0,3.0,1.0,2019-04-30,40.0,NaN,31,2018-07-01
2,201806,GD406715,4,5.0,XXX,C02,F,2016-03-01,NaT,CA1,...,5.416667,5.0,7.0,3.0,1.0,2019-04-30,37.0,NaN,27,2018-06-01
3,201901,GD659982,9,10.0,XXXXX,C02,F,2018-06-10,NaT,CA2,...,7.800000,7.5,10.0,5.0,1.0,2019-04-30,10.0,NaN,6,2019-01-01
4,201811,HD184011,7,6.0,XXXXXX,C02,F,2017-05-01,NaT,CA1,...,6.166667,6.0,8.0,4.0,1.0,2019-04-30,23.0,NaN,18,2018-11-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3941,201902,TS645212,4,2.0,XXXX,C03,F,2018-03-01,2019-03-31,CA1,...,4.500000,4.5,7.0,1.0,0.0,2019-03-31,12.0,2019-02-28 00:00:00,11,2019-02-01
3942,201902,TS741703,5,6.0,XXXX,C03,M,2018-12-08,2019-03-31,CA3,...,6.250000,6.0,8.0,5.0,0.0,2019-03-31,3.0,2019-02-28 00:00:00,1,2019-02-01
3943,201902,TS859258,1,3.0,XXXXX,C02,F,2018-12-07,2019-03-31,CA3,...,2.500000,2.0,5.0,1.0,0.0,2019-03-31,3.0,2019-02-28 00:00:00,1,2019-02-01
3944,201902,TS886985,5,3.0,XXX,C02,F,2018-03-01,2019-03-31,CA1,...,4.250000,4.0,7.0,2.0,1.0,2019-03-31,12.0,2019-02-28 00:00:00,11,2019-02-01


In [67]:
predict_data.isnull().sum()

year_month              0
customer_id             0
count_0                 0
count_1               243
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2842
period                  0
now_date                0
dtype: int64

5. 데이터 전처리하기

In [68]:
predict_data = predict_data.dropna(subset=['count_1'])
predict_data.isna().sum()

year_month              0
customer_id             0
count_0                 0
count_1                 0
name                    0
class                   0
gender                  0
start_date              0
end_date             2651
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2651
period                  0
now_date                0
dtype: int64

In [76]:
predict_data

,year_month,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,period,now_date
0,201808,GD885447,4,6.0,XXXX,C02,F,2017-07-01,NaT,CA2,...,6.083333,6.0,9.0,3.0,1.0,2019-04-30,21.0,NaN,13,2018-08-01
1,201807,HI518408,7,4.0,XXXX,C01,M,2015-12-01,NaT,CA1,...,5.333333,5.0,8.0,3.0,1.0,2019-04-30,40.0,NaN,31,2018-07-01
2,201806,GD406715,4,5.0,XXX,C02,F,2016-03-01,NaT,CA1,...,5.416667,5.0,7.0,3.0,1.0,2019-04-30,37.0,NaN,27,2018-06-01
3,201901,GD659982,9,10.0,XXXXX,C02,F,2018-06-10,NaT,CA2,...,7.800000,7.5,10.0,5.0,1.0,2019-04-30,10.0,NaN,6,2019-01-01
4,201811,HD184011,7,6.0,XXXXXX,C02,F,2017-05-01,NaT,CA1,...,6.166667,6.0,8.0,4.0,1.0,2019-04-30,23.0,NaN,18,2018-11-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3941,201902,TS645212,4,2.0,XXXX,C03,F,2018-03-01,2019-03-31,CA1,...,4.500000,4.5,7.0,1.0,0.0,2019-03-31,12.0,2019-02-28 00:00:00,11,2019-02-01
3942,201902,TS741703,5,6.0,XXXX,C03,M,2018-12-08,2019-03-31,CA3,...,6.250000,6.0,8.0,5.0,0.0,2019-03-31,3.0,2019-02-28 00:00:00,1,2019-02-01
3943,201902,TS859258,1,3.0,XXXXX,C02,F,2018-12-07,2019-03-31,CA3,...,2.500000,2.0,5.0,1.0,0.0,2019-03-31,3.0,2019-02-28 00:00:00,1,2019-02-01
3944,201902,TS886985,5,3.0,XXX,C02,F,2018-03-01,2019-03-31,CA1,...,4.250000,4.0,7.0,2.0,1.0,2019-03-31,12.0,2019-02-28 00:00:00,11,2019-02-01


In [71]:
# object 타입 컬럼만 선택
object_cols = predict_data.select_dtypes(include='object').columns
print(object_cols)


Index(['year_month', 'customer_id', 'name', 'class', 'gender', 'campaign_id',
       'class_name', 'campaign_name', 'calc_date', 'exit_date'],
      dtype='object')


In [ ]:
predict_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3703 entries, 0 to 3945
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   year_month         3703 non-null   object        
 1   customer_id        3703 non-null   object        
 2   count_0            3703 non-null   int64         
 3   count_1            3703 non-null   float64       
 4   name               3703 non-null   object        
 5   class              3703 non-null   object        
 6   gender             3703 non-null   object        
 7   start_date         3703 non-null   datetime64[ns]
 8   end_date           1052 non-null   datetime64[ns]
 9   campaign_id        3703 non-null   object        
 10  is_deleted         3703 non-null   float64       
 11  class_name         3703 non-null   object        
 12  price              3703 non-null   float64       
 13  campaign_name      3703 non-null   object        
 14  mean         

In [83]:
target_col = ['campaign_name','class_name','gender','count_1','routine_flg','period','is_deleted']
predict_data = predict_data[target_col]

In [85]:
predict_data

,campaign_name,class_name,gender,count_1,routine_flg,period,is_deleted
0,0_입회비반액할인,1_주간,F,6.0,1.0,13,0.0
1,2_일반,0_종일,M,4.0,1.0,31,0.0
2,2_일반,1_주간,F,5.0,1.0,27,0.0
3,0_입회비반액할인,1_주간,F,10.0,1.0,6,0.0
4,2_일반,1_주간,F,6.0,1.0,18,0.0
...,...,...,...,...,...,...,...
3941,2_일반,2_야간,F,2.0,0.0,11,1.0
3942,1_입회비무료,2_야간,M,6.0,0.0,1,1.0
3943,1_입회비무료,1_주간,F,3.0,0.0,1,1.0
3944,2_일반,1_주간,F,3.0,1.0,11,1.0


In [88]:
predict_data= pd.get_dummies(predict_data)

In [89]:
predict_data

,count_1,routine_flg,period,is_deleted,campaign_name_0_입회비반액할인,campaign_name_1_입회비무료,campaign_name_2_일반,class_name_0_종일,class_name_1_주간,class_name_2_야간,gender_F,gender_M
0,6.0,1.0,13,0.0,True,False,False,False,True,False,True,False
1,4.0,1.0,31,0.0,False,False,True,True,False,False,False,True
2,5.0,1.0,27,0.0,False,False,True,False,True,False,True,False
3,10.0,1.0,6,0.0,True,False,False,False,True,False,True,False
4,6.0,1.0,18,0.0,False,False,True,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...
3941,2.0,0.0,11,1.0,False,False,True,False,False,True,True,False
3942,6.0,0.0,1,1.0,False,True,False,False,False,True,False,True
3943,3.0,0.0,1,1.0,False,True,False,False,True,False,True,False
3944,3.0,1.0,11,1.0,False,False,True,False,True,False,True,False


7. 모델 구축하기

In [92]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

exit = predict_data.loc[predict_data['is_deleted']==1]
conti = predict_data.loc[predict_data['is_deleted']==0].sample(len(exit))

X = pd.concat([exit, conti],ignore_index=True)
y = X['is_deleted']
del X['is_deleted']

X_train, X_test, y_train, y_test = train_test_split(X, y)

model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)

pred = model.predict(X_test)
print(pred)

[1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0.
 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0.
 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1.
 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1.
 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1.
 1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 1.
 1. 0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0.
 0. 1. 1. 1. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0.
 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0.
 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0. 0.
 0. 1. 1. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0.
 1. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1.
 0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0. 0. 1.
 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 1.

8. 모델 성능 평가하기

In [94]:
model.score(X_train, y_train)

0.9841571609632446

In [95]:
model.score(X_test, pred)

1.0

9.모델에 기여하는 변수 확인하기

In [96]:
importance = pd.DataFrame({'feature_names':X.columns, 'coefficient':model.feature_importances_})
importance

,feature_names,coefficient
0,count_1,0.341203
1,routine_flg,0.119504
2,period,0.458413
3,campaign_name_0_입회비반액할인,0.011545
4,campaign_name_1_입회비무료,0.010398
5,class_name_0_종일,0.008227
6,class_name_1_주간,0.008440
7,class_name_2_야간,0.013163
8,gender_F,0.014711
9,gender_M,0.014396
